In [ ]:
import torch
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch import optim
from torch import nn
from torch.utils.data import DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
#Hyperparameters
input_size = 784  # 28x28 pixels (not directly used in CNN)
num_classes = 10  # digits 0-9
learning_rate = 0.001
batch_size = 64
num_epochs = 10  # Reduced for demonstration purposes

In [ ]:
list = [(1, 2), (2, 3), (3, 4)]
#Enumerate creates pairs of index / value.
for a, b in enumerate(list):
    print(a)

for a, b in enumerate(list):
    print(b)

In [ ]:
#Downloaded the dataset right to Jseid628 folder
train_dataset = datasets.MNIST(root="dataset/", download=True, train=True, transform=transforms.ToTensor())
#The DataLoader takes a data set and a batch size and creates an iterable ("train_loader" here). Each element of the iterable is a batch
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root="dataset/", download=True, train=False, transform=transforms.ToTensor())
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
for batch_idx, (data, targets) in enumerate(train_loader):
    print(f"Batch {batch_idx} | Data shape: {data.shape}, Targets Shape: {targets.shape}")
    if batch_idx == 0:  # Just show first batch
        break

train_features, train_labels = next(iter(train_loader))
img = train_features[0].squeeze()
label = train_labels[0]
plt.imshow(img, cmap="gray")
plt.show()
print(f"Label: {label}")

In [ ]:
#test loader consists of many batches of 64 28*28 images (each x in test_loader is such a batch) and the labels of those images (each y in test_loader is a list of correponding labels)
for x, y in test_loader:
    print(x.shape, y.shape)
    print(x[0].shape)
    print("x.size(0): " + str(x.size(0)))
    print("y:" + str(y))

In [ ]:
#Neural Network Definition
class CNN(nn.Module):
    def __init__(self, in_channels, num_classes = 10):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=8, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        #The pooling layer downsamples by half each time. Initially 28*28 -> 14*14 -> 7*7 due to the kernel size of 2 and stride 2.
        self.fc1 = nn.Linear(16*7*7, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
       
        x = x.reshape(x.shape[0], -1) #flattens x into shape (batch_size, 16*7*7)
        x = self.fc1(x)
        return x
        
model = CNN(in_channels=1, num_classes=num_classes)

In [ ]:
#First layer kernels
first_conv_layer = model.conv1
kernels = first_conv_layer.weight.data.clone()
print(kernels[0])
print(f'kernel shape: {kernels.shape}')

#Second layer kernels
second_conv_layer = model.conv2
kernels2 = second_conv_layer.weight.data.clone()
print(f'kernel shape: {kernels2.shape}')

In [ ]:
#Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
#Training
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
        outputs = model(data)
        loss = criterion(outputs, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if ((epoch % 1)== 0):
        print(f"Epoch {epoch} Loss: {loss.item()}")
    

In [ ]:
#Test
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            outputs = model(x)
            #print(len(outputs)) #should be 64 (batch size)
            max_values, predictions = torch.max(outputs, 1)

            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
    
        accuracy = float(num_correct) / float(num_samples) * 100
        print(f"Got {num_correct}/{num_samples} with accuracy {accuracy:.2f}%")

    model.train()

check_accuracy(test_loader, model)
